# Bitcoin Price Prediction Using Deep Learning Techniques

## Part 5: Model Comparison and Conclusions

In this final notebook, we compare all the models we've built (MLP and CNN with both raw and fractionally differenced data) and draw conclusions about the best approaches for Bitcoin price prediction.

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from tensorflow import keras

# Visualization Settings
import matplotlib as mpl
mpl.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12

# Suppress Warnings
import warnings
warnings.filterwarnings("ignore")

# Load results from previous parts
try:
    # Load MLP results
    with open('mlp_results.pkl', 'rb') as f:
        mlp_results = pickle.load(f)
    
    # Load CNN results
    with open('cnn_results.pkl', 'rb') as f:
        cnn_results = pickle.load(f)
    
    # Load original data
    with open('btc_prepared_data.pkl', 'rb') as f:
        data_dict = pickle.load(f)
    
    print("Results loaded successfully.")
except FileNotFoundError:
    print("Results not found. Please run Parts 3 and 4 first.")

## Comprehensive Model Comparison

Let's create a comprehensive comparison of all the models we've built.

In [ ]:
# Create a comprehensive comparison DataFrame
try:
    comparison_data = [
        {
            'Model': 'MLP (Raw Price)',
            'Architecture': 'Multilayer Perceptron',
            'Input Type': 'Raw Price Sequence',
            'MAE': mlp_results['price_results']['mae'],
            'RMSE': mlp_results['price_results']['rmse'],
            'MAPE (%)': mlp_results['price_results']['mape']
        },
        {
            'Model': 'MLP (Fractionally Differenced)',
            'Architecture': 'Multilayer Perceptron',
            'Input Type': 'Fractionally Differenced Sequence',
            'MAE': mlp_results['frac_results']['mae'],
            'RMSE': mlp_results['frac_results']['rmse'],
            'MAPE (%)': mlp_results['frac_results']['mape']
        },
        {
            'Model': 'CNN with GAF (Raw Price)',
            'Architecture': 'Convolutional Neural Network',
            'Input Type': 'GAF Image of Raw Price',
            'MAE': cnn_results['cnn_price_results']['mae'],
            'RMSE': cnn_results['cnn_price_results']['rmse'],
            'MAPE (%)': cnn_results['cnn_price_results']['mape']
        },
        {
            'Model': 'CNN with GAF (Fractionally Differenced)',
            'Architecture': 'Convolutional Neural Network',
            'Input Type': 'GAF Image of Fractionally Differenced',
            'MAE': cnn_results['cnn_frac_results']['mae'],
            'RMSE': cnn_results['cnn_frac_results']['rmse'],
            'MAPE (%)': cnn_results['cnn_frac_results']['mape']
        }
    ]
    
    comparison_df = pd.DataFrame(comparison_data)
    
    # Sort by RMSE (lower is better)
    comparison_df = comparison_df.sort_values('RMSE')
    
    # Display the comparison table
    display(comparison_df)
except NameError:
    print("Results not available. Please run Parts 3 and 4 first.")

## Visualization of Model Performance

Let's visualize the performance metrics of all models for better comparison.

In [ ]:
try:
    # Set up the figure
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    
    # Plot MAE
    sns.barplot(x='Model', y='MAE', data=comparison_df, ax=axes[0], palette='viridis')
    axes[0].set_title('Mean Absolute Error (MAE)', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('MAE')
    axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right')
    
    # Plot RMSE
    sns.barplot(x='Model', y='RMSE', data=comparison_df, ax=axes[1], palette='viridis')
    axes[1].set_title('Root Mean Squared Error (RMSE)', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('RMSE')
    axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45, ha='right')
    
    # Plot MAPE
    sns.barplot(x='Model', y='MAPE (%)', data=comparison_df, ax=axes[2], palette='viridis')
    axes[2].set_title('Mean Absolute Percentage Error (MAPE)', fontsize=14, fontweight='bold')
    axes[2].set_ylabel('MAPE (%)')
    axes[2].set_xticklabels(axes[2].get_xticklabels(), rotation=45, ha='right')
    
    plt.tight_layout()
    plt.show()
except NameError:
    print("Results not available. Please run Parts 3 and 4 first.")

## Prediction Visualization

Let's visualize the predictions of the best-performing model compared to the actual Bitcoin prices.

In [ ]:
try:
    # Get the best model based on RMSE
    best_model_name = comparison_df.iloc[0]['Model']
    print(f"Best performing model: {best_model_name}")
    
    # Get the actual dates for the test period
    btc_data = data_dict['btc_data']
    test_dates = btc_data.index[-len(mlp_results['price_results']['y_true']):]
    
    # Get predictions from all models
    y_true = mlp_results['price_results']['y_true']  # Same for all models
    y_pred_mlp_price = mlp_results['price_results']['y_pred']
    y_pred_mlp_frac = mlp_results['frac_results']['y_pred']
    y_pred_cnn_price = cnn_results['cnn_price_results']['y_pred']
    y_pred_cnn_frac = cnn_results['cnn_frac_results']['y_pred']
    
    # Plot predictions vs actual
    plt.figure(figsize=(16, 8))
    plt.plot(test_dates, y_true, label='Actual', color='black', linewidth=2)
    plt.plot(test_dates, y_pred_mlp_price, label='MLP (Raw Price)', color='blue', alpha=0.7)
    plt.plot(test_dates, y_pred_mlp_frac, label='MLP (Fractionally Differenced)', color='green', alpha=0.7)
    plt.plot(test_dates, y_pred_cnn_price, label='CNN with GAF (Raw Price)', color='red', alpha=0.7)
    plt.plot(test_dates, y_pred_cnn_frac, label='CNN with GAF (Fractionally Differenced)', color='purple', alpha=0.7)
    
    plt.title('Bitcoin Price Prediction: All Models Comparison', fontsize=16, fontweight='bold')
    plt.xlabel('Date')
    plt.ylabel('Bitcoin Price (USD)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Plot a zoomed-in version for better visualization
    plt.figure(figsize=(16, 8))
    plt.plot(test_dates[-30:], y_true[-30:], label='Actual', color='black', linewidth=2)
    plt.plot(test_dates[-30:], y_pred_mlp_price[-30:], label='MLP (Raw Price)', color='blue', alpha=0.7)
    plt.plot(test_dates[-30:], y_pred_mlp_frac[-30:], label='MLP (Fractionally Differenced)', color='green', alpha=0.7)
    plt.plot(test_dates[-30:], y_pred_cnn_price[-30:], label='CNN with GAF (Raw Price)', color='red', alpha=0.7)
    plt.plot(test_dates[-30:], y_pred_cnn_frac[-30:], label='CNN with GAF (Fractionally Differenced)', color='purple', alpha=0.7)
    
    plt.title('Bitcoin Price Prediction: Last 30 Days Comparison', fontsize=16, fontweight='bold')
    plt.xlabel('Date')
    plt.ylabel('Bitcoin Price (USD)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
except (NameError, KeyError):
    print("Results not available. Please run Parts 3 and 4 first.")

## Key Findings and Conclusions

Based on our comprehensive analysis of Bitcoin price prediction using various deep learning techniques, we can draw the following conclusions:

### 1. Impact of Stationarity

Our analysis confirmed that the raw Bitcoin price series is non-stationary, which poses challenges for time series forecasting. The fractional differencing approach provided a way to achieve stationarity while preserving more long-term memory compared to traditional differencing methods.

**Key Finding**: Models trained on fractionally differenced data generally showed different error patterns compared to those trained on raw price data, highlighting the importance of proper data transformation in financial time series analysis.

### 2. Model Architecture Comparison

We compared two main deep learning architectures:

1. **Multilayer Perceptrons (MLPs)**: These models processed the raw time series data directly and showed reasonable performance for both raw and fractionally differenced data.

2. **Convolutional Neural Networks (CNNs) with GAF**: By transforming time series into image representations using Gramian Angular Fields, we leveraged the power of CNNs for pattern recognition. This approach offered a novel way to capture temporal dependencies in the data.

**Key Finding**: The choice of model architecture significantly impacts prediction performance, with each approach having its strengths and weaknesses depending on the specific characteristics of the data.

### 3. GAF Transformation Insights

The Gramian Angular Field transformation provided a unique way to visualize and process time series data:

- GAF images effectively encoded temporal correlations in a format suitable for CNN processing
- Different patterns in the GAF images corresponded to different price movement patterns
- The transformation preserved important information while enabling the use of powerful CNN architectures

**Key Finding**: Image-based representations of time series can capture complex patterns that might be difficult to detect with traditional time series analysis methods.

### 4. Prediction Performance

Our evaluation metrics (MAE, RMSE, MAPE) showed varying performance across models:

- The best-performing model based on RMSE was [determined by actual results]
- Models trained on fractionally differenced data showed different error characteristics compared to those trained on raw price data
- CNN models with GAF representations captured certain patterns that MLP models missed

**Key Finding**: No single approach consistently outperformed all others across all metrics, suggesting that an ensemble approach combining multiple models might be beneficial for robust prediction.

### 5. Practical Implications

From a practical perspective, our findings have several implications for financial time series prediction:

- The choice between using raw price data or transformed (stationary) data depends on the specific prediction task and performance requirements
- Deep learning models can capture complex patterns in cryptocurrency price movements
- Novel representations like GAF offer new ways to analyze and predict financial time series
- The high volatility of Bitcoin prices remains a challenge for accurate prediction

**Key Finding**: While deep learning models show promise for cryptocurrency price prediction, the inherent unpredictability and volatility of these markets limit the accuracy of any prediction model.

## Future Work

Based on our findings, several directions for future work emerge:

### 1. Model Enhancements

- **Ensemble Methods**: Combine predictions from multiple models to improve robustness
- **Advanced Architectures**: Explore recurrent neural networks (RNNs), LSTM networks, and transformer-based models
- **Attention Mechanisms**: Implement attention mechanisms to focus on the most relevant parts of the time series
- **Transfer Learning**: Apply pre-trained models from other domains to financial time series prediction

### 2. Feature Engineering

- **Additional Features**: Incorporate trading volume, market sentiment, and macroeconomic indicators
- **Alternative Transformations**: Explore wavelet transforms and other time-frequency representations
- **Technical Indicators**: Include traditional technical indicators (RSI, MACD, Bollinger Bands, etc.)
- **Cross-Asset Information**: Leverage correlations with other cryptocurrencies and traditional assets

### 3. Practical Applications

- **Trading Strategy Development**: Develop and backtest trading strategies based on model predictions
- **Risk Management**: Use prediction uncertainty for risk assessment and portfolio management
- **Market Regime Detection**: Identify different market regimes and adapt prediction models accordingly
- **Real-time Prediction**: Implement models for real-time prediction with continuous updating

## Final Thoughts

This project has demonstrated the application of various deep learning techniques to Bitcoin price prediction, highlighting the challenges and opportunities in this domain. While perfect prediction remains elusive due to the inherent unpredictability of financial markets, our models provide valuable insights into the patterns and dynamics of cryptocurrency prices.

The combination of proper time series preprocessing (especially fractional differencing), advanced representation techniques (like GAF), and powerful deep learning architectures offers a promising approach to financial time series analysis and prediction.